# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

from api_keys import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv("../output_data/weather_data.csv", encoding="utf-8")
cities = cities.drop("Unnamed: 0", 1)

cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bengkulu,100,ID,1580354402,47,-3.80,102.27,84.54,5.46
1,George Town,20,MY,1580354131,66,5.41,100.34,86.00,6.93
2,Demba,100,CD,1580354404,98,-5.51,22.27,69.37,2.62
3,Banjar,75,ID,1580354405,84,-8.19,114.97,87.80,11.41
4,Tasiilaq,91,GL,1580354406,68,65.61,-37.64,28.40,31.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]


In [6]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heatmap_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Iquique,0,CL,1580354422,60,-20.22,-70.14,75.20,10.29
1,Vao,0,NC,1580354453,73,-22.67,167.48,77.07,14.52
2,Saint-Louis,0,SN,1580354477,15,16.33,-15.00,78.15,13.24
3,Cockburn Town,0,TC,1580354498,66,21.46,-71.14,77.59,3.89
4,Codrington,0,AU,1580354521,44,-38.27,141.97,76.53,8.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Iquique,0,CL,1580354422,60,-20.22,-70.14,75.20,10.29,Iquique
1,Vao,0,NC,1580354453,73,-22.67,167.48,77.07,14.52,Vao
2,Saint-Louis,0,SN,1580354477,15,16.33,-15.00,78.15,13.24,Bélel Davo
3,Cockburn Town,0,TC,1580354498,66,21.46,-71.14,77.59,3.89,Cockburn Town
4,Codrington,0,AU,1580354521,44,-38.27,141.97,76.53,8.48,Yambuk


In [9]:
narrowed_city_df['Hotel Name']= narrowed_city_df
narrowed_city_df.dropna()
narrowed_city_df.to_csv('hotel_export.csv')

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display Map
fig


Figure(layout=FigureLayout(height='420px'))